# Setup

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sympy import *
from promoter_solving_core import *

In [2]:
log_tpm_df = pd.read_csv('../data/precise_1.0/log_tpm.csv', index_col = 0)
log_tpm_df.head()

,control__wt_glc__1,control__wt_glc__2,fur__wt_dpd__1,fur__wt_dpd__2,fur__wt_fe__1,fur__wt_fe__2,fur__delfur_dpd__1,fur__delfur_dpd__2,fur__delfur_fe2__1,fur__delfur_fe2__2,...,efeU__menFentC_ale29__1,efeU__menFentC_ale29__2,efeU__menFentC_ale30__1,efeU__menFentC_ale30__2,efeU__menFentCubiC_ale36__1,efeU__menFentCubiC_ale36__2,efeU__menFentCubiC_ale37__1,efeU__menFentCubiC_ale37__2,efeU__menFentCubiC_ale38__1,efeU__menFentCubiC_ale38__2
b0002,11.006653,11.130196,11.704951,11.888217,11.064810,10.779071,9.976401,10.291135,11.229767,11.214065,...,10.271327,10.276565,11.148538,11.170578,11.676604,11.726097,11.881529,11.923237,11.496410,11.552762
b0003,10.415654,10.523138,11.423835,11.803781,10.776984,10.597810,9.596833,10.191503,10.897938,10.861157,...,10.160291,10.116861,10.314322,10.392251,10.916426,10.909277,11.023924,11.038426,10.624301,10.764195
b0004,10.594871,10.725062,10.457270,10.779162,10.394971,10.113950,8.741617,9.082231,10.185151,10.164655,...,10.475069,10.434352,10.679541,10.723953,11.143310,11.112721,11.184795,11.241845,10.953206,11.001006
b0005,6.316102,6.258499,5.422129,5.336121,6.716069,6.410864,4.626949,4.756153,6.527653,6.136168,...,5.979079,5.705586,6.306120,6.291340,5.058537,4.835550,5.448097,5.757951,5.873964,5.808618
b0006,6.915771,6.897597,6.775645,6.782605,6.761813,6.816532,6.686767,6.860036,6.862147,6.817480,...,8.371287,8.322390,8.137515,8.071837,7.354131,7.365536,7.328101,7.314761,8.057750,8.105213


In [3]:
# Check that column adds to 10**7
total_tpm = 0
for gene in log_tpm_df.index:
    if not pd.isna(log_tpm_df.loc[gene,'control__wt_glc__1']):
        total_tpm = total_tpm + 2**log_tpm_df.loc[gene,'control__wt_glc__1']

print(total_tpm)

1003922.9999997448


# Sympy Stuff

In [4]:
default = sympify('Eq((KeqOpening*kEscape*promoterConcVal)/((KdRNAP/RNAP+1+KeqOpening+KdRNAP/RNAP*TF/k_d_TF)*(u+kDeg)),mRNA)')
default

Eq(KeqOpening*kEscape*promoterConcVal/((kDeg + u)*(KdRNAP*TF/(RNAP*k_d_TF) + KdRNAP/RNAP + KeqOpening + 1)), mRNA)

In [5]:
# Define constants and convert to symbols
t_half_life_deg = 300

constants = {
    'k_d_TF': 1, # May change depending on model
    'RNAP': 10**-6, # Concentration of RNAP
    'TF': 0, # Concentration of TF
    'u': 1/3600, # Growth Rate
    'kDeg': np.log(2)/t_half_life_deg, # Rate of degradation
    'promoterConcVal': 10**-9, # Promoter concentration
    #'cActivator': 0,
    #'KdRNAPCrp': 1,
    'mRNA_total': 1800, # Total mRNA/cell from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3554401
    'cell_volume': 10**-15, # Liters from https://bionumbers.hms.harvard.edu/bionumber.aspx?id=100004&ver=19
}

In [6]:
# NOTE: If you use other parameters in the equation, you need to update promoter_solving_core @ calculate_minimum_escape
log_ranges = {
    'KdRNAP': [-7,-5],
    #'KdRNAPCrp': [-9,-7],
    'kEscape': [-3,1],
    #'cActivator': [-1.3010299956639813, -0.6989700043360187],
}

In [7]:
gene = 'b1101'
condition = 'crp__delar1ar2_glyc__2'
lambda_df, k_df = create_grid(gene_exp = [2**log_tpm_df.loc[gene,condition]],# list
                              gene_name = [gene],# list
                              equation = default,
                              constants = constants,
                              num_steps = 5,
                              **log_ranges)

In [8]:
lambda_df

,equation,lambda,order
KdRNAP,[2.0e-27*(193179348761631.0*KeqOpening*kEscape...,<function _lambdifygenerated at 0x16ade1000>,"(KeqOpening, kEscape, mRNA)"
KeqOpening,[5.0e+20*mRNA*(1000000.0*KdRNAP + 1.0)/(193179...,<function _lambdifygenerated at 0x16af76680>,"(KdRNAP, kEscape, mRNA)"
kEscape,[2588268.37964426*mRNA*(1000000.0*KdRNAP + Keq...,<function _lambdifygenerated at 0x16af74a60>,"(KdRNAP, KeqOpening, mRNA)"
mRNA,[3.86358697523262e-7*KeqOpening*kEscape/(10000...,<function _lambdifygenerated at 0x16af77250>,"(KdRNAP, KeqOpening, kEscape)"


In [9]:
k_df

,log10_mRNA_target,parameter_grid
gene_name,,
b1101,-9.388925,"[(-7.0, -2.9305929702838305, 0.999999999999999..."


# Testing more complicated sympy equations
Need to update promoter_solving_core calculate minimum escape if you use these